<a href="https://colab.research.google.com/github/palienko105/Projects/blob/main/SQL_Ships_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!add-apt-repository -y ppa:sergey-dryabzhinsky/packages
!apt update
!apt install sqlite3

In [1]:
# MENU: Runtime > Restart runtime
import sqlite3
sqlite3.sqlite_version  # '3.38.0'

'3.40.1'

In [2]:
import pandas as pd
import numpy as np

In [3]:
con = sqlite3.connect('BD_Ships')
cur = con.cursor()

In [4]:
def select(sql):
  return pd.read_sql(sql,con)

# Удаляем таблицы, если они есть в БД

In [11]:
#Удаляем таблицу 
cur.execute('''
drop table if exists Battles
''')
cur.execute('''
drop table if exists Classes
''')
cur.execute('''
drop table if exists Ships
''')
cur.execute('''
drop table if exists Outcomes
''')
con.commit() 

# Создаем таблицы в БД "BD_Ships"

In [12]:
cur.execute('''
CREATE TABLE Battles (
	name varchar (20) NOT NULL ,
	date datetime NOT NULL 
)
''')

cur.execute('''
CREATE TABLE Classes (
	class varchar (50) NOT NULL ,
	type varchar (2) NOT NULL ,
	country varchar (20) NOT NULL ,
	numGuns int NULL ,
	bore real NULL ,
	displacement int NULL 
)
''')

cur.execute('''
CREATE TABLE Ships (
	name varchar (50) NOT NULL ,
	class varchar (50) NOT NULL ,
	launched int NULL 
)
''')

cur.execute('''
CREATE TABLE Outcomes (
	ship varchar (50) NOT NULL ,
	battle varchar (20) NOT NULL ,
	result varchar (10) NOT NULL 
)
''')
con.commit() 

In [13]:
sql = '''
SELECT name FROM sqlite_master WHERE type='table'
'''
select(sql)

,name
0,Battles
1,Classes
2,Ships
3,Outcomes


# Добавляем данные в таблицы

In [14]:
cur.execute('''
insert into Classes values
('Bismarck','bb','Germany',8,15,42000),
('Iowa','bb','USA',9,16,46000),
('Kongo','bc','Japan',8,14,32000),
('North Carolina','bb','USA',12,16,37000),
('Renown','bc','Gt.Britain',6,15,32000),
('Revenge','bb','Gt.Britain',8,15,29000),
('Tennessee','bb','USA',12,14,32000),
('Yamato','bb','Japan',9,18,65000)
''')

cur.execute('''
insert into Battles values
('Guadalcanal','1942-11-15 00:00:00'),
('North Atlantic','1941-05-25 00:00:00'),
('North Cape','1943-12-26 00:00:00'),
('Surigao Strait','1944-10-25 00:00:00'),
('#Cuba62a'   , '1962-10-20'),
('#Cuba62b'   , '1962-10-25')
''')

cur.execute('''
insert into Ships values
('California','Tennessee',1921),
('Haruna','Kongo',1916),
('Hiei','Kongo',1914),
('Iowa','Iowa',1943),
('Kirishima','Kongo',1915),
('Kongo','Kongo',1913),
('Missouri','Iowa',1944),
('Musashi','Yamato',1942),
('New Jersey','Iowa',1943),
('North Carolina','North Carolina',1941),
('Ramillies','Revenge',1917),
('Renown','Renown',1916),
('Repulse','Renown',1916),
('Resolution','Renown',1916),
('Revenge','Revenge',1916),
('Royal Oak','Revenge',1916),
('Royal Sovereign','Revenge',1916),
('Tennessee','Tennessee',1920),
('Washington','North Carolina',1941),
('Wisconsin','Iowa',1944),
('Yamato','Yamato',1941),
('South Dakota','North Carolina',1941)
''')

cur.execute('''
insert into Outcomes values
('Bismarck','North Atlantic','sunk'),
('California','Surigao Strait','OK'),
('Duke of York','North Cape','OK'),
('Fuso','Surigao Strait','sunk'),
('Hood','North Atlantic','sunk'),
('King George V','North Atlantic','OK'),
('Kirishima','Guadalcanal','sunk'),
('Prince of Wales','North Atlantic','damaged'),
('Rodney','North Atlantic','OK'),
('Schamhorst','North Cape','sunk'),
('South Dakota','Guadalcanal','damaged'),
('Tennessee','Surigao Strait','OK'),
('Washington','Guadalcanal','OK'),
('West Virginia','Surigao Strait','OK'),
('Yamashiro','Surigao Strait','sunk'),
('California','Guadalcanal','damaged')
''')

con.commit()

# Описание БД:

"Корабли"

Рассматривается БД кораблей, участвовавших во второй мировой войне. Имеются следующие отношения:    
Classes (class, type, country, numGuns, bore, displacement)     
Ships (name, class, launched)   
Battles (name, date)    
Outcomes (ship, battle, result)    
Корабли в «классах» построены по одному и тому же проекту, и классу присваивается либо имя первого корабля, построенного по данному проекту, либо названию класса дается имя проекта, которое не совпадает ни с одним из кораблей в БД. Корабль, давший название классу, называется головным.
Отношение Classes содержит имя класса, тип (bb для боевого (линейного) корабля или bc для боевого крейсера), страну, в которой построен корабль, число главных орудий, калибр орудий (диаметр ствола орудия в дюймах) и водоизмещение ( вес в тоннах). В отношении Ships записаны название корабля, имя его класса и год спуска на воду. В отношение Battles включены название и дата битвы, в которой участвовали корабли, а в отношении Outcomes – результат участия данного корабля в битве (потоплен-sunk, поврежден - damaged или невредим - OK).    
Замечания. 1) В отношение Outcomes могут входить корабли, отсутствующие в отношении Ships. 2) Потопленный корабль в последующих битвах участия не принимает.

# Задание:

Одной из характеристик корабля является половина куба калибра его главных орудий (mw). С точностью до 2 десятичных знаков определите среднее значение mw для кораблей каждой страны, у которой есть корабли в базе данных.

In [39]:
sql = '''
with vrem1 as -- Корабли, которые есть Outcomes, но нет в Ships
( 
Select ship
from Outcomes
EXCEPT 
select name
from Ships
),

vrem2 as --Находим такие корабли из vrem1, по которым есть инфа в Classes
(
Select class
from Classes
INTERSECT
Select *
from vrem1
),

vrem3 as --все корабли, по которым есть информация о калибре их главных орудий
(
select name, class
from Ships
UNION
Select class as name, class
from vrem2
)

--Формирую итоговую выгрузку по запросу:

select country, ROUND(avg(bore*bore*bore/2), 2) as weight
from vrem3 v3
join Classes c 
  on v3.class=c.class
group by country
'''
select (sql)

,country,weight
0,Germany,1687.50
1,Gt.Britain,1687.50
2,Japan,1886.67
3,USA,1897.78
